# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122314e+02     1.525664e+00
 * time: 0.12708616256713867
     1     1.092813e+01     9.533364e-01
 * time: 0.9932291507720947
     2    -1.164608e+01     9.663196e-01
 * time: 1.1322221755981445
     3    -3.434668e+01     7.519069e-01
 * time: 1.2879440784454346
     4    -4.795908e+01     5.197166e-01
 * time: 1.4402990341186523
     5    -5.719416e+01     1.749408e-01
 * time: 1.5920400619506836
     6    -5.994368e+01     1.138361e-01
 * time: 1.7102751731872559
     7    -6.094494e+01     6.038248e-02
 * time: 1.8259050846099854
     8    -6.132397e+01     6.203168e-02
 * time: 1.9499061107635498
     9    -6.162173e+01     3.827157e-02
 * time: 2.0731420516967773
    10    -6.183176e+01     2.746989e-02
 * time: 2.1874091625213623
    11    -6.199512e+01     1.925088e-02
 * time: 2.309467077255249
    12    -6.206217e+01     1.785457e-02
 * time: 2.4289469718933105
    13    -6.212964e+01     1.749902e-02
 * time: 2.5408570766

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671075
    AtomicLocal         -18.8557683
    AtomicNonlocal      14.8522639
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485379 
    Xc                  -19.3336823

    total               -62.261666461279
